# Illustrationen der Gartenlaube

Im Volltexttranskript der Gartenlaube sind Illustrationen freigestellt eingebettet. Dieses Skript analyisiert alle vorhandenen Artikeleinheiten in Wikisource und extrahiert freigestellte Gartenlaube-Illustrationien.

## 1. Extraktion der Illustrations-Dateinamen
Identifizierung von eingebetteten Illustrationen in Gartenlaube-Artikel via parse-API


## 2. Überprüfung ob die Datei in Commons bereits besteht
Bei der Artikel-Erstellung in Commons werden Illustrationen oft bereits "gesetzt" ohne, dass diese freigestellt in Commons gespeichert sind. (Darstellung als Rotlink) - Für die weitere Verarbeitung in diesem Skript ist aber notwendig, dass die Bilder physisch existieren - daher wie commons-Parse-API Überprüfung, ob die Datei existiert, wenn ja, dann wird die Medien-ID (pageid) und der Commons-File-Name für die Weiterverarbeitung übernommen.

## 3. Extraktion der Bildlegende aus Wikisource
Ein Großteil der Gartenlaube-Illustrationen ist mit einer Legende versehen. Dieser Textbaustein liegt in Wikisource nicht in der Artikel-Seite sondern im Seiten-Namespace des jeweiligen Scans, in der speziellen Wikisource-Vorlage ```{{BRU}}``` 


In [20]:
import requests
import re
import json
import urllib
import os
from SPARQLWrapper import SPARQLWrapper, JSON
import time
import sys
import wikitextparser as wtp

endpoint_url = "https://query.wikidata.org/sparql"

wdImgQSFile = 'output/img_WD_'+str(time.time())[0:10]+'.tsv'
sdcStmtQSFile = 'output/img_SDC_Stmt_'+str(time.time())[0:10]+'.tsv'
sdcCaptionQSFile = 'output/img_SDC_Caption_'+str(time.time())[0:10]+'.tsv'
#QSFile = 'output/qs'+str(time.time())[0:10]+'.tsv'
try:
    os.remove(wdImgQSFile)
    os.remove(sdcQSFile)
except:
    pass

S = requests.Session()
artikelCounter = 1
resultSet = []

def getWikibaseIDviaMwapi(pagetitle,URL):
    param_wbase = { 'action' : 'query', 'prop' : 'pageprops', 'ppprop': 'wikibase_item',
             'redirects': 1, 'titles': pagetitle,'format': "json" }
    R = S.get(url=URL, params=param_wbase)
    DATA = R.json()
    try:
        for page in DATA["query"]["pages"]:
            #print(page)
            try:
                return DATA["query"]["pages"][page]["pageprops"]["wikibase_item"]
            except KeyError:
                pass
    except KeyError:
        pass

    
def getWikibaseIDviaSPARQL(pagetitle):
    query = """SELECT * WHERE {
      ?gartenlaubeArtikel wdt:P1433 wd:Q655617.
      ?article schema:about ?gartenlaubeArtikel.
      ?article schema:isPartOf <https://de.wikisource.org/>.
      ?article schema:name ?pageTitle.
      FILTER(STR(?pageTitle)='"""+str(pagetitle)+"""')
    }"""
    sparql = SPARQLWrapper(endpoint_url)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()
    #results = get_results(endpoint_url, query)
    #for result in results["results"]["bindings"]:
        #print(result["gartenlaubeArtikel"]["value"])

    
def getOldIDviaMwapi(pagetitle):
    param_wbase = { 'action' : 'query', 'prop' : 'info',
             'redirects': 1, 'titles': pagetitle,'format': "json" }
    R = S.get(url=URL, params=param_wbase)
    DATA = R.json()
    for page in DATA["query"]["pages"]:
        #print(page)
        try:
            return DATA["query"]["pages"][page]["lastrevid"]
        except KeyError:
            pass   

def parseWikiAPI(URL,PARAMS):
    R = S.get(url=URL, params=PARAMS)
    DATA = R.json()
    return DATA


#Run through Category
def runthroughCat(DATA,count=0):
    try:
        while DATA['query']:
            #print(str(artikelCounter))
            #print(resultSet)
            global artikelCounter
            artikelCounter = ExtractImages(DATA,artikelCounter)

            try:
                #print(DATA['continue']['cmcontinue'])
                PARAMS['cmcontinue'] = DATA['continue']['cmcontinue']
                DATA = parseWikiAPI(URL,PARAMS)
            except KeyError:
                break
    except KeyError:
        pass


#Parse via Subcategories
def runthroughSubCat(kategorie):
    param_SubCat = PARAMS
    param_SubCat["cmtype"]="subcat"
    param_SubCat["cmtitle"] = kategorie
    DATACAT = parseWikiAPI(URL,param_SubCat)
    #print(DATACAT["query"])
    try:
        #print(str(artikelCounter))
        #print(resultSet)
        for subcat in DATACAT["query"]["categorymembers"]:
            print (">>=============================================")
            print (subcat["title"])
            PARAMS["cmtitle"] = subcat["title"]
            PARAMS.pop('cmtype',None)
            #print(PARAMS)
            DATA = parseWikiAPI(URL,PARAMS)
            runthroughCat(DATA)
    except KeyError:
        pass
    

def get_results(endpoint_url, query):
    user_agent = "Wikisource:Die_Gartenlaube Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()    


URL = "https://de.wikisource.org/w/api.php"
#?action=query&list=categorymembers&cmlimit=500&cmtitle=Kategorie:Album_der_S%C3%A4chsischen_Industrie&format=json
PARAMS = {
    'action': "query",
    'list': "categorymembers",
    'cmtitle': "Kategorie:Die_Gartenlaube_(1853)_Artikel",
    'cmlimit': 500,
    'format': "json"
}

def ExtractImages(DATA,artikelCounter):
    #print(DATA)
    for artikel in DATA["query"]["categorymembers"]:
        param_text = { 'action' : 'parse', 'page': artikel["title"], 'prop':'properties|images', 'format': "json" }
        R = S.get(url=URL, params=param_text)
        imgdata = R.json()

        qid = ""
        mid = ""
        for pprops in imgdata["parse"]["properties"]:
            if pprops["name"] == "wikibase_item":
                qid = pprops["*"]

        for image in imgdata["parse"]["images"]:
            f_wd = open(wdImgQSFile,'a')
            f_sdcStmt = open(sdcStmtQSFile,'a')
            f_sdcCap = open(sdcCaptionQSFile,'a')
            #print(image)
            regex = r"Die_Gartenlaube_\(([\d]{4})\).*_b_([\d]{0,3}).*.jpg"
            matches = re.finditer(regex, image, re.MULTILINE)

            for matchNum, match in enumerate(matches, start=1):
                param_text = {'action' : 'parse', 'page': "File:"+image, 'prop':'images', 'format': "json" }
                try:
                    R = S.get(url="http://commons.wikimedia.org/w/api.php", params=param_text)
                    commonsdata = R.json()
                except:
                    time.sleep(60)
                    print(" -- wait for API --")
                    continue
                
                
                #print(commonsdata)
                try:
                    commonsTitle =re.sub("File:","",commonsdata["parse"]["title"])
                    mid = "M"+str(commonsdata["parse"]["pageid"] )
                except KeyError:
                    print("image "+image+" not in Commons yet")
                    break
                #print(image+" || page: "+match.group(2)+" || "+mid)
                #SDC Statement01 - published in 'GArtenlaube' generell
                print(mid+"\tP1433\tQ655617\tP304\t\""+match.group(2)+"\"\tP577\t+"+match.group(1)+"-00-00T00:00:00Z/9",file=f_sdcStmt)
                #SDC Statement02 - published in -> Artikel-Item
                print(mid+"\tP1433\t"+qid,file=f_sdcStmt)
                print(mid+"\tP1433\tQ655617\tP304\t\""+match.group(2)+"\"\tP577\t+"+match.group(1)+"-00-00T00:00:00Z/9")


                #Wikisource Seite 
                # https://de.wikisource.org/w/api.php?action=parse&page=Seite:Die_Gartenlaube_(1897)_476.jpg&format=json&prop=wikitext
                #extract aus BRU-Vorlage die Bildunterschrift
                seitenPage = "Seite:Die Gartenlaube ("+match.group(1)+") "+match.group(2)+".jpg"
                param_text = { 'action' : 'parse', 'page': seitenPage, 'prop':'wikitext', 'format': "json" }
                R = S.get(url=URL, params=param_text)
                wsSeitenData = R.json()
                parse = wtp.parse(wsSeitenData["parse"]["wikitext"]["*"])
                legende = ""
                for temp in parse.templates:

                    if temp.name == "BRU" and re.sub(" ","_",temp.arguments[0].value) == image:
                        #print(temp.arguments[0].value)

                        #print (temp.arguments[3].value)
                        '''
                        legende = re.sub("<br />"," ",temp.arguments[3].value)
                        legende = legende.replace('\n', ' ').replace('\r', '')
                        legende = re.sub("<br>"," ",legende)
                        '''
                        s = str(temp.arguments[3].value).replace('<br/>',' ').replace('<br />',' ').replace('<br>',' ').replace('\n', ' ').replace('\r', '')
                        wtp.remove_markup(s)
                        legende = wtp.parse(s).plain_text()

                        if legende.strip() != "" and len(legende.strip())<200:
                            legendeQS = "\"\""+legende.strip()+"\"\" "
                        else:
                            legendeQS = ""
                        sdcDe =  legendeQS+"Illustration in: Die Gartenlaube ("+match.group(1)+"), S. "+match.group(2)
                        sdcEn =  "illustration published in German magazine Die Gartenlaube ("+match.group(1)+"), p. "+match.group(2)
                        print(sdcDe)
                        #remove_markup(temp.arguments[3].value)
                        #SDC-Statement 03 -> deDescirptions ("caption")
                        print(mid+",\""+sdcDe+"\",\""+sdcEn+"\"",file=f_sdcCap)

                #query qid by given filename: https://w.wiki/36c$
                #nachschauen ob Bild bereits gesetzt, wenn nicht QID P18 Filname
                query = """SELECT * WHERE {
                      BIND(IRI(CONCAT("http://commons.wikimedia.org/wiki/Special:FilePath/", ENCODE_FOR_URI(\""""+re.sub("File:","",commonsdata["parse"]["title"])+"""\"))) AS ?img)
                      ?item p:P18 ?imgStmt. ?imgStmt ps:P18 ?img.
                    }"""
                #print(query)
                results = get_results(endpoint_url, query)
                if len(results["results"]["bindings"]) > 0:
                    for result in results["results"]["bindings"]:
                        print(result["item"]["value"])
                else:
                    if legende.strip() != "":
                        legendeStmt = "\tP2096\tde:\""+legende+"\""
                    else:
                        legendeStmt = ""
                    print(qid+"\tP18\t\""+commonsTitle+"\""+legendeStmt,file=f_wd)
                    print("image not linked in Wikidata-Item "+qid)
                print("==")
                #time.sleep(0.1)

DATA = parseWikiAPI(URL,PARAMS)
#print(DATA)
count = 0

R = S.get(url=URL, params=PARAMS)
DATA = R.json()


runthroughSubCat("Kategorie:Die_Gartenlaube_Artikel")



>>=============================================
Kategorie:Die Gartenlaube (1853) Artikel
M13102887	P1433	Q655617	P304	"397"	P577	+1853-00-00T00:00:00Z/9
""Alexander von Humboldt."" Illustration in: Die Gartenlaube (1853), S. 397
http://www.wikidata.org/entity/Q19125952
==
M13271474	P1433	Q655617	P304	"162"	P577	+1853-00-00T00:00:00Z/9
Illustration in: Die Gartenlaube (1853), S. 162
http://www.wikidata.org/entity/Q19186871
==
M16115186	P1433	Q655617	P304	"549"	P577	+1853-00-00T00:00:00Z/9
""Das Kapitol in Washington."" Illustration in: Die Gartenlaube (1853), S. 549
http://www.wikidata.org/entity/Q19127933
==
M16115192	P1433	Q655617	P304	"550"	P577	+1853-00-00T00:00:00Z/9
""Der Indianerhäuptling bei seiner Ankunft in Washington."" Illustration in: Die Gartenlaube (1853), S. 550
http://www.wikidata.org/entity/Q19127933
==
M16115196	P1433	Q655617	P304	"550"	P577	+1853-00-00T00:00:00Z/9
""Der Indianerhäuptling bei seinem Weggange von Washington."" Illustration in: Die Gartenlaube (1853), S

M13271608	P1433	Q655617	P304	"192"	P577	+1853-00-00T00:00:00Z/9
Illustration in: Die Gartenlaube (1853), S. 192
http://www.wikidata.org/entity/Q19215356
==
M16104045	P1433	Q655617	P304	"318"	P577	+1853-00-00T00:00:00Z/9
Illustration in: Die Gartenlaube (1853), S. 318
http://www.wikidata.org/entity/Q19215447
http://www.wikidata.org/entity/Q19143799
==
M16115041	P1433	Q655617	P304	"467"	P577	+1853-00-00T00:00:00Z/9
""Karte des russisch-türkischen Kriegsschauplatzes."" Illustration in: Die Gartenlaube (1853), S. 467
http://www.wikidata.org/entity/Q19148832
==
M13370387	P1433	Q655617	P304	"300"	P577	+1853-00-00T00:00:00Z/9
""Das Goldwaschen mit dem Longtom."" Illustration in: Die Gartenlaube (1853), S. 300
http://www.wikidata.org/entity/Q19148965
==
M13370450	P1433	Q655617	P304	"301"	P577	+1853-00-00T00:00:00Z/9
""Erste Untersuchung des Bodens."" Illustration in: Die Gartenlaube (1853), S. 301
http://www.wikidata.org/entity/Q19148965
==
M13370527	P1433	Q655617	P304	"301"	P577	+1853-00-00T0

Illustration in: Die Gartenlaube (1853), S. 072
http://www.wikidata.org/entity/Q19153276
==
M13270328	P1433	Q655617	P304	"073"	P577	+1853-00-00T00:00:00Z/9
Illustration in: Die Gartenlaube (1853), S. 073
http://www.wikidata.org/entity/Q19153276
==
M13270593	P1433	Q655617	P304	"112"	P577	+1853-00-00T00:00:00Z/9
""Vinzenz Prießnitz."" Illustration in: Die Gartenlaube (1853), S. 112
http://www.wikidata.org/entity/Q19237366
==
M13267021	P1433	Q655617	P304	"113"	P577	+1853-00-00T00:00:00Z/9
""J. H. Rausse."" Illustration in: Die Gartenlaube (1853), S. 113
http://www.wikidata.org/entity/Q1597266
==
M16104067	P1433	Q655617	P304	"347"	P577	+1853-00-00T00:00:00Z/9
""Bukarescht."" Illustration in: Die Gartenlaube (1853), S. 347
http://www.wikidata.org/entity/Q19237753
==
M13267848	P1433	Q655617	P304	"022"	P577	+1853-00-00T00:00:00Z/9
Illustration in: Die Gartenlaube (1853), S. 022
http://www.wikidata.org/entity/Q19238419
http://www.wikidata.org/entity/Q104776900
==
M16104070	P1433	Q655617	P304	"

M23193708	P1433	Q655617	P304	"361"	P577	+1854-00-00T00:00:00Z/9
""Eduard Duller."" Illustration in: Die Gartenlaube (1854), S. 361
http://www.wikidata.org/entity/Q19187979
==
M23193631	P1433	Q655617	P304	"313"	P577	+1854-00-00T00:00:00Z/9
Illustration in: Die Gartenlaube (1854), S. 313
http://www.wikidata.org/entity/Q19182926
==
M23184098	P1433	Q655617	P304	"164"	P577	+1854-00-00T00:00:00Z/9
""Charles Napier."" Illustration in: Die Gartenlaube (1854), S. 164
http://www.wikidata.org/entity/Q19182930
==
M23184111	P1433	Q655617	P304	"165"	P577	+1854-00-00T00:00:00Z/9
""Empfang fremder Offiziere im Lager bei Kalefat."" Illustration in: Die Gartenlaube (1854), S. 165
http://www.wikidata.org/entity/Q19182930
==
M23195402	P1433	Q655617	P304	"537"	P577	+1854-00-00T00:00:00Z/9
""Der Herzog von Cambridge.General Brown.Canrobert.St. Arnaud.Prinz Napoleon. Lord Raglan."" Illustration in: Die Gartenlaube (1854), S. 537
http://www.wikidata.org/entity/Q19182913
==
M23124744	P1433	Q655617	P304	"055"	P

http://www.wikidata.org/entity/Q19170287
==
M18720103	P1433	Q655617	P304	"476"	P577	+1854-00-00T00:00:00Z/9
""Aeußere Ansicht der neuen Synagoge in Leipzig."" Illustration in: Die Gartenlaube (1854), S. 476
http://www.wikidata.org/entity/Q19183873
==
M18720150	P1433	Q655617	P304	"477"	P577	+1854-00-00T00:00:00Z/9
""Innere Ansicht der neuen Synagoge in Leipzig."" Illustration in: Die Gartenlaube (1854), S. 477
http://www.wikidata.org/entity/Q19183873
==
M23193624	P1433	Q655617	P304	"307"	P577	+1854-00-00T00:00:00Z/9
""Halifax."" Illustration in: Die Gartenlaube (1854), S. 307
http://www.wikidata.org/entity/Q19221424
http://www.wikidata.org/entity/Q98807688
==
M23184330	P1433	Q655617	P304	"231"	P577	+1854-00-00T00:00:00Z/9
""Kronstadt."" Illustration in: Die Gartenlaube (1854), S. 231
http://www.wikidata.org/entity/Q19221981
http://www.wikidata.org/entity/Q79847479
==
M23195675	P1433	Q655617	P304	"640"	P577	+1854-00-00T00:00:00Z/9
""Das Panoptikon."" Illustration in: Die Gartenlaube (185

...

M24598447	P1433	Q655617	P304	"016"	P577	+1897-00-00T00:00:00Z/9
""Elektrische Brennschere."" Illustration in: Die Gartenlaube (1897), S. 016
http://www.wikidata.org/entity/Q19174199
==
M24598464	P1433	Q655617	P304	"016"	P577	+1897-00-00T00:00:00Z/9
""Elektrische Bratpfanne."" Illustration in: Die Gartenlaube (1897), S. 016
http://www.wikidata.org/entity/Q19174199
==
M24598483	P1433	Q655617	P304	"018"	P577	+1897-00-00T00:00:00Z/9
""Elektrisches Bügeleisen."" Illustration in: Die Gartenlaube (1897), S. 018
http://www.wikidata.org/entity/Q19174199
==
M9596047	P1433	Q655617	P304	"548"	P577	+1897-00-00T00:00:00Z/9
""Elefanten-Schildkröten im Hamburger Zoologischen Garten. Nach einer Skizze gezeichnet von U. Specht."" Illustration in: Die Gartenlaube (1897), S. 548
http://www.wikidata.org/entity/Q764058
http://www.wikidata.org/entity/Q62617118
==
image Die_Gartenlaube_(1897)_b_227.jpg not in Commons yet
image Die_Gartenlaube_(1897)_b_652_1.jpg not in Commons yet
image Die_Gartenlaube_(1897)_

image Die_Gartenlaube_(1897)_b_250.jpg not in Commons yet
image Die_Gartenlaube_(1897)_b_251_1.jpg not in Commons yet
image Die_Gartenlaube_(1897)_b_251_2.jpg not in Commons yet
M17998758	P1433	Q655617	P304	"253"	P577	+1897-00-00T00:00:00Z/9
""Der Kaskadenfall in der Kuhflucht bei Partenkirchen. Nach dem Gemälde von F. Feldhütter."" Illustration in: Die Gartenlaube (1897), S. 253
http://www.wikidata.org/entity/Q62617224
==
image Die_Gartenlaube_(1897)_b_337.jpg not in Commons yet
image Die_Gartenlaube_(1897)_b_589.jpg not in Commons yet
image Die_Gartenlaube_(1897)_b_305.jpg not in Commons yet
image Die_Gartenlaube_(1897)_b_676.jpg not in Commons yet
image Die_Gartenlaube_(1897)_b_245.jpg not in Commons yet
image Die_Gartenlaube_(1897)_b_853.jpg not in Commons yet
M5772510	P1433	Q655617	P304	"277"	P577	+1897-00-00T00:00:00Z/9
""Johannes Brahms. Nach der letzten Aufnahme des Hofateliers von R. Krziwanek in Wien."" Illustration in: Die Gartenlaube (1897), S. 277
http://www.wikidata.org/e

image Die_Gartenlaube_(1897)_b_292.jpg not in Commons yet
image Die_Gartenlaube_(1897)_b_364_1.jpg not in Commons yet
image Die_Gartenlaube_(1897)_b_364_2.jpg not in Commons yet
image Die_Gartenlaube_(1897)_b_365.jpg not in Commons yet
M25236021	P1433	Q655617	P304	"366"	P577	+1897-00-00T00:00:00Z/9
""Der Laubmann."" Illustration in: Die Gartenlaube (1897), S. 366
http://www.wikidata.org/entity/Q19223584
==
image Die_Gartenlaube_(1897)_b_211_1.jpg not in Commons yet
image Die_Gartenlaube_(1897)_b_211_2.jpg not in Commons yet
image Die_Gartenlaube_(1897)_b_805.jpg not in Commons yet
M8376110	P1433	Q655617	P304	"041"	P577	+1897-00-00T00:00:00Z/9
""Riesentukane beim Bananenschmaus. Nach einer Originalskizze von F. Specht gezeichnet von A. Specht."" Illustration in: Die Gartenlaube (1897), S. 041
http://www.wikidata.org/entity/Q19225654
==
M24599414	P1433	Q655617	P304	"033"	P577	+1897-00-00T00:00:00Z/9
""’s Ringl. Nach dem Gemälde von E. v. Müller."" Illustration in: Die Gartenlaube (1897),

http://www.wikidata.org/entity/Q62617093
==
image Die_Gartenlaube_(1897)_b_736.jpg not in Commons yet
M13182346	P1433	Q655617	P304	"889"	P577	+1897-00-00T00:00:00Z/9
""Sylvester-Orakel. Nach einer Originalzeichnung von Oscar Gräf."" Illustration in: Die Gartenlaube (1897), S. 889
http://www.wikidata.org/entity/Q19229821
==
M8376355	P1433	Q655617	P304	"281"	P577	+1897-00-00T00:00:00Z/9
""Die Tauben des Venustempels. Nach einem Gemälde von H. Motte."" Illustration in: Die Gartenlaube (1897), S. 281
http://www.wikidata.org/entity/Q62617026
==
image Die_Gartenlaube_(1897)_b_320.jpg not in Commons yet
image Die_Gartenlaube_(1897)_b_596.jpg not in Commons yet
image Die_Gartenlaube_(1897)_b_532.jpg not in Commons yet
image Die_Gartenlaube_(1897)_b_445.jpg not in Commons yet
image Die_Gartenlaube_(1897)_b_641.jpg not in Commons yet
image Die_Gartenlaube_(1897)_b_872_1.jpg not in Commons yet
image Die_Gartenlaube_(1897)_b_872_2.jpg not in Commons yet
M24576376	P1433	Q655617	P304	"005"	P577	+189

M12867220	P1433	Q655617	P304	"284"	P577	+1897-00-00T00:00:00Z/9
""Aussicht auf die Kirche Santa Eufemia."" Illustration in: Die Gartenlaube (1897), S. 284
http://www.wikidata.org/entity/Q19181913
==
M12867139	P1433	Q655617	P304	"286"	P577	+1897-00-00T00:00:00Z/9
""Im Inneren des Aquariums."" Illustration in: Die Gartenlaube (1897), S. 286
http://www.wikidata.org/entity/Q19181913
==
M12867052	P1433	Q655617	P304	"287"	P577	+1897-00-00T00:00:00Z/9
""Der Dampfer „Rudolf Virchow“."" Illustration in: Die Gartenlaube (1897), S. 287
http://www.wikidata.org/entity/Q19181913
==
image Die_Gartenlaube_(1897)_b_165.jpg not in Commons yet
M13238177	P1433	Q655617	P304	"877"	P577	+1897-00-00T00:00:00Z/9
""Zum Jahreswechsel"" Illustration in: Die Gartenlaube (1897), S. 877
http://www.wikidata.org/entity/Q19239924
==
M59564917	P1433	Q655617	P304	"285"	P577	+1897-00-00T00:00:00Z/9
Illustration in: Die Gartenlaube (1897), S. 285
http://www.wikidata.org/entity/Q16740628
==
M27131301	P1433	Q655617	P304	"012

In [19]:
import requests
import re
import time
import wikitextparser as wtp
import sys
from SPARQLWrapper import SPARQLWrapper, JSON

S = requests.Session()
URL = "https://de.wikisource.org/w/api.php"
endpoint_url = "https://query.wikidata.org/sparql"

wdImgQSFile = 'output/img_WD_'+str(time.time())[0:10]+'.tsv'
sdcQSFile = 'output/img_SDC_'+str(time.time())[0:10]+'.tsv'
f_wd = open(wdImgQSFile,'w')
f_sdc = open(sdcQSFile,'w')

#Skizzen aus dem Imkerleben (alles fetig außer SDC)
#Das Pellerhaus in Nürnberg (keine Bilder freigstellt)
#Das Raimundtheater in Wien (1 Bild nicht in WD)
#Schiller’s Räuber auf ihrer ersten Bühne (zweizweilige Legende)
#Ein auferstandenes Wikingerschiff (Legende enthält "")
param_text = { 'action' : 'parse', 'page': "Das Raimundtheater in Wien", 'prop':'properties|images', 'format': "json" }
R = S.get(url=URL, params=param_text)
imgdata = R.json()


# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

'''
results = get_results(endpoint_url, query)

for result in results["results"]["bindings"]:
    print(result)
'''

for pprops in imgdata["parse"]["properties"]:
    if pprops["name"] == "wikibase_item":
        qid = pprops["*"]

for image in imgdata["parse"]["images"]:
    
    #f_sdc = open(sdcQSFile,'a')
    #print(image)
    regex = r"Die_Gartenlaube_\(([\d]{4})\).*_b_([\d]{0,3}).*.jpg"
    matches = re.finditer(regex, image, re.MULTILINE)

    for matchNum, match in enumerate(matches, start=1):
        param_text = {'action' : 'parse', 'page': "File:"+image, 'prop':'images', 'format': "json" }
        R = S.get(url="http://commons.wikimedia.org/w/api.php", params=param_text)
        commonsdata = R.json()
        #print(commonsdata)
        try:
            commonsTitle =re.sub("File:","",commonsdata["parse"]["title"])
            mid = "M"+str(commonsdata["parse"]["pageid"] )
        except KeyError:
            print("image "+image+" not in Commons yet")
            break
        #print(image+" || page: "+match.group(2)+" || "+mid)
        #SDC Statement01 - published in 'GArtenlaube' generell
        print(mid+"\tP1433\tQ655617\tP304\t\""+match.group(2)+"\"\tP577\t+"+match.group(1)+"-00-00T00:00:00Z/9",file=f_sdc)
        #SDC Statement02 - published in -> Artikel-Item
        print(mid+"\tP1433\t"+qid,file=f_sdc)
        print(mid+"\tP1433\tQ655617\tP304\t\""+match.group(2)+"\"\tP577\t+"+match.group(1)+"-00-00T00:00:00Z/9")
        

        #Wikisource Seite 
        # https://de.wikisource.org/w/api.php?action=parse&page=Seite:Die_Gartenlaube_(1897)_476.jpg&format=json&prop=wikitext
        #extract aus BRU-Vorlage die Bildunterschrift
        seitenPage = "Seite:Die Gartenlaube ("+match.group(1)+") "+match.group(2)+".jpg"
        param_text = { 'action' : 'parse', 'page': seitenPage, 'prop':'wikitext', 'format': "json" }
        R = S.get(url=URL, params=param_text)
        wsSeitenData = R.json()
        parse = wtp.parse(wsSeitenData["parse"]["wikitext"]["*"])
        legende = ""
        for temp in parse.templates:
            
            if temp.name == "BRU" and re.sub(" ","_",temp.arguments[0].value) == image:
                #print(temp.arguments[0].value)
                
                #print (temp.arguments[3].value)
                '''
                legende = re.sub("<br />"," ",temp.arguments[3].value)
                legende = legende.replace('\n', ' ').replace('\r', '')
                legende = re.sub("<br>"," ",legende)
                '''
                #s = str(temp.arguments[3].value).replace('\n', ' ').replace('\r', '')
                #print(str(temp.arguments[3].value))
                s = str(temp.arguments[3].value).replace('<br/>',' ').replace('<br />',' ').replace('<br>',' ').replace('\n', ' ').replace('\r', '')
                print("---")
                #print(s)
                print("---")
                wtp.remove_markup(s)
                legende = wtp.parse(s).plain_text()
                #legende = s
                if legende.strip() != "" and len(legende.strip())<200:
                    legendeQS = "\"\""+legende.strip()+"\"\" "
                else:
                    legendeQS = ""
                sdcDe =  legendeQS+"Illustration in: Die Gartenlaube ("+match.group(1)+"), S. "+match.group(2)
                sdcEn =  "illustration published in German magazine Die Gartenlaube ("+match.group(1)+"), p. "+match.group(2)
                print(sdcDe)
                #remove_markup(temp.arguments[3].value)
                #SDC-Statement 03 -> deDescirptions ("caption")
                print(mid+"\tDde\t\""+sdcDe+"\"",file=f_sdc)

        #query qid by given filename: https://w.wiki/36c$
        #nachschauen ob Bild bereits gesetzt, wenn nicht QID P18 Filname
        query = """SELECT * WHERE {
              BIND(IRI(CONCAT("http://commons.wikimedia.org/wiki/Special:FilePath/", ENCODE_FOR_URI(\""""+re.sub("File:","",commonsdata["parse"]["title"])+"""\"))) AS ?img)
              ?item p:P18 ?imgStmt. ?imgStmt ps:P18 ?img.
            }"""
        #print(query)
        results = get_results(endpoint_url, query)
        if len(results["results"]["bindings"]) > 0:
            for result in results["results"]["bindings"]:
                print(result["item"]["value"])
        else:
            if legende.strip() != "":
                legendeStmt = "\tP2096\tde:\""+legende+"\""
            else:
                legendeStmt = ""
            print(qid+"\tP18\t\""+commonsTitle+"\""+legendeStmt,file=f_wd)
            print("image not linked in Wikidata-Item "+qid)
        print("==")

print("",file=f_wd)

M93996106	P1433	Q655617	P304	"875"	P577	+1893-00-00T00:00:00Z/9
---
---
""Das Raimundtheater in Wien."" Illustration in: Die Gartenlaube (1893), S. 875
http://www.wikidata.org/entity/Q99155673
==
M101612346	P1433	Q655617	P304	"876"	P577	+1893-00-00T00:00:00Z/9
---
---
""Adam Müller-Guttenbrunn,   der Direktor des Raimundtheaters.    Nach einer Photographie von Joh. E. Hahn in Wien."" Illustration in: Die Gartenlaube (1893), S. 876
http://www.wikidata.org/entity/Q99155673
==
